# Cleaner
The goal of this notebook is to combine and clean the data.

We will procceed as follows: 

-Load the data from CSV files  
-Clean the data to fix any errors or inconsistencies  
-Encode the statistical data type (numeric, categorical)  
-Impute any missing values  


In [1]:
import pandas as pd
import numpy as np

The first function just reads the data off the csv files. Notice that we have two folders. One contains houses separated by the year they were built and the other houses based on the attributes/filters, for example autonomous_heating, they possess. Here is a list of all the possible filters.

['autonomous_heating', 'central_heating', 'individual_heating', 'no_heating', 'petrol_heating', 'natural_gas_heating', 'LPG_heating', 'electrical_heating', 'thermal_storage_heating', 'wood_headting', 'pellet_heating', 'heat_pump_heating', 'with_AC', 'with_storage_room', 'with_elavator', 'with_solar_heater', 'with_fireplace', 'Furnished', 'with_parking', 'with_garden', 'with_pool', 'with_balcony', 'last_floor']

In [2]:
def load_file(year=None,attribute=None):#reads data from a csv file and puts it in dataframe. Attribute can be any of the above listed.
    if attribute==None:
        df0=pd.read_csv(f"Houses_data/Houses_built_in_{year}.csv",header=0,index_col=0)
    elif year==None:
        df0=pd.read_csv(f"Houses_data_filters/Houses_data_{attribute}/All_combined.csv",header=0,index_col=0)
    else:
        print("You must specify either year or attribute, not both.")

    return(df0)


First we read all the csv files produced by the scraper and we combine them in a dataframe. We then drop any extra columns, reset the indexing and drop any entries with n\a values.

In [3]:
def load_years():
    df0=load_file(year=1952)
    for i in range(1953,2023):
        df1=load_file(i)
        df0=pd.concat([df0,df1],ignore_index=True)
    return df0
df0=load_years()

In [4]:
df0.isna().sum()

Location                64
Price                   64
Total_area              64
House_type              64
Floor                   64
Rooms                   64
Bathrooms               64
submission_date         64
Year_of_construction     0
dtype: int64

We see that our dataframe contains N/A values so we drop these.

In [5]:
def clean(df):
    df=df.dropna()
    df.reset_index(drop=True,inplace=True)
    df.drop_duplicates(inplace=True)
    return df
df_clean=clean(df0)

/tmp/ipykernel_98823/438433548.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


We check that indeed 64 rows were dropped.

In [6]:
print(df_clean.shape,df0.shape)

(24314, 9) (24378, 9)


Next we read all the houses having the different attributes. Our strategy will be to check for each entry in df_clean whether or not it appears in those dataframes. If it does the column named with that attribute will take the value 1 otherwise it will be zero.

In [7]:
def load_attributes():
    
    filters=['autonomous_heating', 'central_heating', 'individual_heating', 'no_heating', 
            'petrol_heating', 'natural_gas_heating', 'LPG_heating', 'electrical_heating', 
            'thermal_storage_heating', 'wood_headting', 'pellet_heating', 'heat_pump_heating', 
            'with_AC', 'with_storage_room', 'with_elavator', 'with_solar_heater', 'with_fireplace', 
            'Furnished', 'with_parking', 'with_garden', 'with_pool', 'with_balcony', 'last_floor']
    dict_filters={}
    for filter in filters:
        df_filter=load_file(attribute=filter).drop('submission_date', axis=1)
        df_filter.reset_index(inplace=True)
        df_filter.drop_duplicates(inplace=True)
        dict_filters[filter]=df_filter
    return dict_filters
dict_filters=load_attributes()

Now for each entry in our df_clean we must check if this entry appears in each of the dataframes of the dictionary dict_filters.


The next snippet checks, for each filter, whether each row in the main DataFrame matches a row in the corresponding filter DataFrame. It then adds a new column to df_clean for each filter, marking 1 if the filter applies to that row and 0 otherwise. This enables easy analysis of which attributes apply to each house entry.

In [8]:
def merge_filters(df_clean, dict_filters):
    df_clean_mod=df_clean.loc[:, ['Location', 'Price', 'Total_area', 'House_type', 'Floor', 'Rooms',"Bathrooms"]]
    for filter in dict_filters.keys():
        df_filter=dict_filters[filter]
        merged=df_clean_mod.merge(df_filter, how="left",indicator=True)
        df_clean.loc[:,filter]=(merged["_merge"]=="both").astype(int)
    return df_clean
df_clean=merge_filters(df_clean, dict_filters)

/tmp/ipykernel_98823/1488292245.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.loc[:,filter]=(merged["_merge"]=="both").astype(int)
/tmp/ipykernel_98823/1488292245.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.loc[:,filter]=(merged["_merge"]=="both").astype(int)
/tmp/ipykernel_98823/1488292245.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [9]:
df_clean

,Location,Price,Total_area,House_type,Floor,Rooms,Bathrooms,submission_date,Year_of_construction,autonomous_heating,...,with_storage_room,with_elavator,with_solar_heater,with_fireplace,Furnished,with_parking,with_garden,with_pool,with_balcony,last_floor
0,Βαρδάρης - Λαχανόκηποι (Θεσσαλονίκη - Δήμος),€245.000,70τ.μ.,Διαμέρισμα,7,2.0,1.0,07/06/2025,1952,1,...,0,1,1,0,1,0,0,0,0,1
1,Άγιος Δημήτριος (Κέντρο Θεσσαλονίκης),€123.000,30τ.μ.,Studio / Γκαρσονιέρα,4,1.0,1.0,07/06/2025,1952,1,...,0,1,0,0,1,0,0,0,1,0
2,Ρωμαϊκή Αγορά - Βενιζέλου - Ιστορικό Κέντρο (Κ...,€173.000,39τ.μ.,Studio / Γκαρσονιέρα,4,1.0,1.0,07/06/2025,1952,0,...,0,0,0,0,0,0,0,0,1,0
3,Διοικητήριο (Κέντρο Θεσσαλονίκης),€132.000,30τ.μ.,Studio / Γκαρσονιέρα,4,1.0,1.0,07/06/2025,1952,0,...,0,1,0,0,1,0,0,0,0,0
4,Διοικητήριο (Κέντρο Θεσσαλονίκης),€173.000,39τ.μ.,Studio / Γκαρσονιέρα,4,1.0,1.0,07/06/2025,1952,1,...,0,1,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24309,Κάτω Τούμπα (Τούμπα),€496.000,160τ.μ.,Μεζονέτα,5,3.0,1.0,09/02/2024,2022,0,...,0,0,0,1,0,0,0,0,1,1
24310,Δελασάλ (Πεύκα),€260.000,140τ.μ.,Μεζονέτα,1,3.0,1.0,28/11/2023,2022,1,...,1,0,0,0,0,1,0,0,1,0
24311,Άνωθεν Περιφερειακού (Εύοσμος),€240.000,135τ.μ.,Μεζονέτα,4,3.0,1.0,27/01/2025,2022,0,...,0,0,0,0,0,1,0,0,1,1
24312,Επέκταση (Πυλαία),€500.000,270τ.μ.,Μονοκατοικία,4,5.0,1.0,17/10/2024,2022,1,...,1,1,0,1,0,1,0,0,1,1


We find the set of all possible locations for the houses on sale.

In [10]:
df_clean["Location"].unique()

array(['Βαρδάρης - Λαχανόκηποι (Θεσσαλονίκη - Δήμος)',
       'Άγιος Δημήτριος (Κέντρο Θεσσαλονίκης)',
       'Ρωμαϊκή Αγορά - Βενιζέλου - Ιστορικό Κέντρο (Κέντρο Θεσσαλονίκης)',
       'Διοικητήριο (Κέντρο Θεσσαλονίκης)',
       'Λευκός Πύργος (Κέντρο Θεσσαλονίκης)',
       'Κέντρο Θεσσαλονίκης (Θεσσαλονίκη - Δήμος)',
       'Πλατεία Εμπορίου - Ανω Λαδάδικα - Ιστορικό Κέντρο (Κέντρο Θεσσαλονίκης)',
       'Καμάρα - Ροτόντα (Κέντρο Θεσσαλονίκης)',
       'Αντιγονιδών (Κέντρο Θεσσαλονίκης)',
       'Πλατεία Ναυαρίνου (Κέντρο Θεσσαλονίκης)',
       'Τουρκικό Προξενείο (Κέντρο Θεσσαλονίκης)',
       'ΧΑΝΘ (Κέντρο Θεσσαλονίκης)', 'ΔΕΘ (Κέντρο Θεσσαλονίκης)',
       'Αριστοτέλους - Ιστορικό Κέντρο (Κέντρο Θεσσαλονίκης)',
       'Αγία Σοφία (Κέντρο Θεσσαλονίκης)',
       'Πανεπιστημιούπολη (Κέντρο Θεσσαλονίκης)',
       'Άνω Πόλη (Θεσσαλονίκη - Δήμος)', 'Κουλέ Καφέ (Άνω Πόλη)',
       'Κέντρο (Μενεμένη)', 'Συκιές (Θεσσαλονίκη - Περιφ/κοί δήμοι)',
       'Κάτω Ομόνοια (Σταυρούπολη)', 'Καλλιθέ

We notice that there are too many possible locations with significant overlap. We need to tidy things up. We will create 27 custom locations based on the above list. The choice was made based on the areas visible on google maps. The dict_loc dictionary will contain each of these locations as keys. The values of the dictionary will be a list which contains their corresponding instances in the above list.

In [11]:
dict_loc={"Center_east_of_aristotle_square" : [
    "Λευκός Πύργος (Κέντρο Θεσσαλονίκης)",
    "ΔΕΘ - Πανεπιστήμια (Κέντρο Θεσσαλονίκης)",
   "Καμάρα - Ροτόντα (Κέντρο Θεσσαλονίκης)",
    "Αγία Σοφία (Κέντρο Θεσσαλονίκης)",'ΔΕΘ - Πανεπιστήμια (Κέντρο Θεσσαλονίκης)',
]

,"Vardaris_xirokrini" :[
    "Ξηροκρήνη - Παναγία Φανερωμένη (Θεσσαλονίκη - Δήμος)",'Λαχανόκηποι (Βαρδάρης - Λαχανόκηποι)','Ξηροκρήνη (Ξηροκρήνη - Παναγία Φανερωμένη)', 
    "Σφαγεία - Ιχθυόσκαλα (Θεσσαλονίκη - Δήμος)",'Παναγία Φανερωμένη (Ξηροκρήνη - Παναγία Φανερωμένη)',
    'Βαρδάρης (Βαρδάρης - Λαχανόκηποι)','Βαρδάρης - Λαχανόκηποι (Θεσσαλονίκη - Δήμος)',
    'Παλαιός Σιδηροδρομικός Σταθμός (Σφαγεία - Ιχθυόσκαλα)','Ξηροκρήνη - Παναγία Φανερωμένη (Θεσσαλονίκη - Δήμος)','Σταθμός ΟΣΕ (Βαρδάρης - Λαχανόκηποι)', 
],

"Center_west_of_including_aristotle_square" : [ 'Λαδάδικα (Κέντρο Θεσσαλονίκης)','Κέντρο Θεσσαλονίκης (Θεσσαλονίκη - Δήμος)',
    "Αντιγονιδών (Κέντρο Θεσσαλονίκης)",
     "Ιστορικό Κέντρο (Κέντρο Θεσσαλονίκης)",
    'Άγιος Δημήτριος (Κέντρο Θεσσαλονίκης)',
    "Διοικητήριο (Κέντρο Θεσσαλονίκης)",
    "Άγιος Δημήτριος (Κέντρο Θεσσαλονίκης)",
    "Κέντρο Θεσσαλονίκης (Θεσσαλονίκη - Δήμος)",
     'Διοικητήριο (Κέντρο Θεσσαλονίκης)',
    "Λαδάδικα (Κέντρο Θεσσαλονίκης)"
],

"Voulgari_Ntepo" : [
    'Βούλγαρη - Άγιος Ελευθέριος - Ντεπώ (Βούλγαρη - Ντεπώ - Μαρτίου)',
       "Βούλγαρη - Ντεπώ - Μαρτίου (Θεσσαλονίκη - Δήμος)",'Βούλγαρη - Ντεπώ - Μαρτίου (Θεσσαλονίκη - Δήμος)','Μαρτίου (Βούλγαρη - Ντεπώ - Μαρτίου)', 
],

"Panorama" : [ 'Κέντρο (Πανόραμα)','Συνοικισμός Νόμου 751 (Πανόραμα)','Παλιός Οικισμός Πανοράματος (Πανόραμα)', 'Οικισμός Μακεδονία (Πανόραμα)','Οικισμός Τοπογράφων (Πανόραμα)', 'Πανόραμα (Θεσσαλονίκη - Περιφ/κοί δήμοι)',
    
],

"Agia_triada" : ['Φάληρο - Ιπποκράτειο (Θεσσαλονίκη - Δήμος)','Αγία Τριάδα (Φάληρο - Ιπποκράτειο)','Ιπποκράτειο (Φάληρο - Ιπποκράτειο)', ],

"Ano_poli_Evaggelistria" : ['Τσινάρι (Άνω Πόλη)','Κάστρα (Άνω Πόλη)', 'Άνω Πόλη (Θεσσαλονίκη - Δήμος)', 
    '40 Εκκλησιές (40 Εκκλησιές - Ευαγγελίστρια)',"Άνω Πόλη (Θεσσαλονίκη - Δήμος)", 'Άγιος Παύλος (Θεσσαλονίκη - Περιφ/κοί δήμοι)','Πανεπιστημιούπολη (Άγιος Παύλος)',
    "40 Εκκλησιές - Ευαγγελίστρια (Θεσσαλονίκη - Δήμος)",'Χίλια Δέντρα (Άγιος Παύλος)', 'Γεντί Κουλέ (Άγιος Παύλος)','Ευαγγελίστρια (40 Εκκλησιές - Ευαγγελίστρια)',
],

"Ampelokipoi" : ['Επτάλοφος (Αμπελόκηποι)', 'Σκεπάρνη (Αμπελόκηποι)', 'Μεταγωγών (Αμπελόκηποι)',
     'Αμπελόκηποι (Θεσσαλονίκη - Περιφ/κοί δήμοι)',
       'Βόσπορος (Αμπελόκηποι)','Δημαρχείου (Αμπελόκηποι)', 'Φιλλίπου (Αμπελόκηποι)',"Αμπελόκηποι (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'Ζωοδόχου Πηγής (Αμπελόκηποι)',
       
],

"Sykies" : [ 'Κέντρο (Συκιές)', 'Άγιοι Θεόδωροι (Συκιές)','Άλσος (Συκιές)', 'Δροσιά (Συκιές)', 'Συκιές (Θεσσαλονίκη - Περιφ/κοί δήμοι)',
            'Μαύρος Γάτος (Συκιές)', 'Παράσχου (Συκιές)', 'Καλλιθέα (Συκιές)','Επταπύργιο (Συκιές)', 'Ροδοχώρι (Συκιές)',],

"Menemeni_lachanokipoi" : ['Κέντρο (Μενεμένη)', 
    "Μενεμένη (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'Λαχανόκηποι (Μενεμένη)','Μενεμένη (Θεσσαλονίκη - Περιφ/κοί δήμοι)',
],

"Peuka": ['Κέντρο (Πεύκα)','Πεύκα (Θεσσαλονίκη - Περιφ/κοί δήμοι)','Στροφή Φιλύρου (Πεύκα)','Δελασάλ (Πεύκα)',],

"Evosmos_Kordelio" : ['Κέντρο (Ελευθέριο-Κορδελιό)','Ευαγγελισμός - Νέος Κουκλουτζάς (Εύοσμος)','Εύοσμο (Εύοσμος)',
    'Κέντρο (Εύοσμος)', 'Κάτω Ηλιούπολη (Εύοσμος)', 'Νέο Κορδελιό (Ελευθέριο-Κορδελιό)',"Εύοσμος (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'Νέα Πολιτεία (Εύοσμος)','Άνωθεν Περιφερειακού (Εύοσμος)', 'Ελευθέριο-Κορδελιό (Θεσσαλονίκη - Περιφ/κοί δήμοι)',
       
],

"Stavroupoli" : ['Ομόνοια (Σταυρούπολη)','Άνωθεν Ασύλου (Σταυρούπολη)',
    'Πρόνοια (Σταυρούπολη)','Σταυρούπολη (Θεσσαλονίκη - Περιφ/κοί δήμοι)','Ανθόκηποι (Σταυρούπολη)',"Σταυρούπολη (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'Ακριτών (Σταυρούπολη)', 'Κέντρο (Σταυρούπολη)',  'Αμπελώνες (Σταυρούπολη)', 
       'Τερψιθέα (Σταυρούπολη)','Κάτω Ηλιούπολη (Σταυρούπολη)','Άνω Ηλιούπολη (Σταυρούπολη)',
],

"Neapoli" : ['Νεάπολη (Νεάπολη)','Άνω Αναγέννηση (Νεάπολη)','Πυροπαθών (Νεάπολη)','Ρήγα Φεραίου (Νεάπολη)','Κέντρο (Νεάπολη)','Κάτω Αναγέννηση (Νεάπολη)','Άγιος Γιώργιος - Τρώαδα (Νεάπολη)','Νεάπολη (Θεσσαλονίκη - Περιφ/κοί δήμοι)','Στρεμπενιώτη (Νεάπολη)','Κόκκορας (Νεάπολη)','Κουντουριώτη (Νεάπολη)', 
         ],

"Analipsi" : [     'Ανάληψη (Ανάληψη - Μπότσαρη - Νέα Παραλία)', 
       'Μπότσαρη (Ανάληψη - Μπότσαρη - Νέα Παραλία)', 'Ανάληψη - Μπότσαρη - Νέα Παραλία (Θεσσαλονίκη - Δήμος)','Νέα Παραλία (Ανάληψη - Μπότσαρη - Νέα Παραλία)',],

"Toumpa" : ['Άνω Τούμπα (Τούμπα)','Παπάφη (Τούμπα)',
       'Τούμπα (Θεσσαλονίκη - Δήμος)', 'Μαλακοπή (Τούμπα)','Κάτω Τούμπα (Τούμπα)',],

"Xarilaou" : [ 'Χαριλάου (Θεσσαλονίκη - Δήμος)','Οσία Ξένη (Χαριλάου)',  'Βρυσάκι (Χαριλάου)', 'Αγία Κυριακή (Χαριλάου)','Άγιος Νικόλαος (Χαριλάου)', 'Νέα Ελβετία (Χαριλάου)', ],

"Kalamaria" : [ 'Καραμπουρνακι (Καλαμαριά)','Φοίνικας (Καλαμαριά)', 'Άγιος Ιωάννης (Καλαμαριά)','Νέα Κρήνη (Καλαμαριά)', 'Αρετσού (Καλαμαριά)','Κηφισιά (Καλαμαριά)', 
    'Κουρί (Καλαμαριά)',
       'Κέντρο (Καλαμαριά)', 'Βότση (Καλαμαριά)', 'Βυζάντιο (Καλαμαριά)', 'Καλαμαριά (Θεσσαλονίκη - Περιφ/κοί δήμοι)',"Καλαμαριά (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'Επέκταση (Καλαμαριά)', 'Δέρκων (Καλαμαριά)','Άγιος Παντελεήμων (Καλαμαριά)', 
],

"Pylaia" : [
    'Πυλαία (Θεσσαλονίκη - Περιφ/κοί δήμοι)','Κέντρο (Πυλαία)','Επέκταση (Πυλαία)','Ελαιώνες (Πυλαία)', 'Βίλλα Ριτζ (Πυλαία)','Πυλαιώτικα (Πυλαία)',"Πυλαία (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'Τοπογράφοι (Πυλαία)','Mediterranean Cosmos (Πυλαία)', 'Πουρνάρι (Πυλαία)','Κωνσταντινοπολίτικα (Πυλαία)','Άνω Μαλακοπή (Πυλαία)',
],

"Peraia" : ['Περαία (Θερμαϊκός)',],

"Triandria" : [
    'Δόξα (Τριανδρία - Δόξα)',"Τριανδρία - Δόξα (Θεσσαλονίκη - Δήμος)",'Τριανδρία - Δόξα (Θεσσαλονίκη - Δήμος)', 'Τριανδρία (Τριανδρία - Δόξα)',
],

"Polichni" : [ 'Επέκταση (Πολίχνη)','Κέντρο (Πολίχνη)', 'Μετέωρα (Πολίχνη)','Άγιος Ραφαήλ (Πολίχνη)', 'Καρατάσσου (Πολίχνη)', 'Άγιος Ιωάννης (Πολίχνη)',
            'Άνω Πολίχνη (Πολίχνη)','Άγιος Παντελεήμονας (Πολίχνη)','Πολίχνη (Θεσσαλονίκη - Περιφ/κοί δήμοι)', ],

"Efkarpia_Nikopoli" : [
    'Νικόπολη (Σταυρούπολη)','Κέντρο (Ευκαρπία)',"Ευκαρπία (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'Ευξεινούπολη Ευκαρπίας (Ευκαρπία)',
],

"Oreokastro" : [
    'Κέντρο (Ωραιόκαστρο)','Γαλήνη (Ωραιόκαστρο)', "Ωραιόκαστρο (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'ΒΙ.ΠΑ. (Ωραιόκαστρο)','Παλαιόκαστρο (Ωραιόκαστρο)', 
    'Ασπρόβρυση (Ωραιόκαστρο)', 'Αμφιθέα (Ωραιόκαστρο)','Ωραιόκαστρο (Θεσσαλονίκη - Περιφ/κοί δήμοι)',
],

"Thermi" : [
    "Θέρμη (Θεσσαλονίκη - Περιφ/κοί δήμοι)",'Περιοχή Αεροδρομίου (Θέρμη)','Ταγαράδες (Θέρμη)','Νέα Ραιδεστός (Θέρμη)','Κέντρο (Θέρμη)', 'Νέο Ρύσιο (Θέρμη)', 'Τριάδιο (Θέρμη)',
],

"Sindos_Diavata" : [
    "Σίνδος (Εχέδωρος)",
    "Νέα Μαγνησία (Εχέδωρος)",
    "Καλοχώρι (Εχέδωρος)",
    "Αξιός (Θεσσαλονίκη - Υπόλ. Νομού)",'Διαβατά (Εχέδωρος)',
    
],

"out_of_thessaloniki" : [
    "Κουφάλια (Θεσσαλονίκη - Υπόλ. Νομού)",
    "Χαλκηδόνα (Θεσσαλονίκη - Υπόλ. Νομού)",
    "Άγιος Αθανάσιος (Θεσσαλονίκη - Υπόλ. Νομού)",
    "Κορώνεια (Θεσσαλονίκη - Υπόλ. Νομού)",
    "Λαγκαδάς (Θεσσαλονίκη - Υπόλ. Νομού)",
    "Χορτιάτης (Θεσσαλονίκη - Περιφ/κοί δήμοι)"
]}


Now we create a list which will contain the aproximate location of each entry on our dataframe. We append this list to our dataframe.

In [12]:
def encode_location(df_clean, dict_loc):
    
    loc_aprox=[]
    for j in range(len(df_clean["Location"])):
        other=True
        for i in dict_loc:
            if df_clean["Location"][j] in dict_loc[i]:
                loc_aprox.append(i)
                other=False
        if other==True:
            loc_aprox.append("out_of_thessaloniki")
    df_clean.loc[:,"Location_aprox"]=loc_aprox
    return df_clean
df_clean=encode_location(df_clean, dict_loc)

/tmp/ipykernel_98823/1425738877.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.loc[:,"Location_aprox"]=loc_aprox


We also notice that the floor column has non-numeric values. We need to fix that so we will create a new column with numeric values corresponding to the categorical ones. The correspondance goes as follows ΗΜ means mezzanine so it corresponds to 0.5, ΗΜ/ΥΠ means semi-basement so it corresponds to -0.5, ΙΣ means ground floor so it corresponds to 0 and ΥΠ means basement so it corresponds to -1.

In [13]:
df_clean["Floor"].unique()

array(['7', '4', '3', '6', '8', '1', 'ΗΜ', 'ΗΜ/ΥΠ', 'ΥΠ', '5', '2', 'ΙΣ',
       '10', '9', '18', '16', '12', 3, 2], dtype=object)

We also remove any non numeric characters from our columns.

In [14]:
def clean3(df):

    df.insert(2, "Price_cleaned", df["Price"].str.replace(".", ""))
    df.insert(2, "Price (in €)", df["Price_cleaned"].str.replace("€", ""))
    df.insert(2, "Total_area (square meters)", df["Total_area"].str.replace("τ.μ.", ""))
    df["Floor"]=df["Floor"].replace("ΗΜ",0.5)
    df["Floor"]=df["Floor"].replace("ΗΜ/ΥΠ",-0.5)
    df["Floor"]=df["Floor"].replace("ΙΣ",0)
    df["Floor"]=df["Floor"].replace("ΥΠ",-1)
    df=df.drop(columns=["Price","Price_cleaned","Total_area"])
    return df

df_clean = clean3(df_clean.copy())


In [15]:
df_clean

,Location,Total_area (square meters),Price (in €),House_type,Floor,Rooms,Bathrooms,submission_date,Year_of_construction,autonomous_heating,...,with_elavator,with_solar_heater,with_fireplace,Furnished,with_parking,with_garden,with_pool,with_balcony,last_floor,Location_aprox
0,Βαρδάρης - Λαχανόκηποι (Θεσσαλονίκη - Δήμος),70,245000,Διαμέρισμα,7,2.0,1.0,07/06/2025,1952,1,...,1,1,0,1,0,0,0,0,1,Vardaris_xirokrini
1,Άγιος Δημήτριος (Κέντρο Θεσσαλονίκης),30,123000,Studio / Γκαρσονιέρα,4,1.0,1.0,07/06/2025,1952,1,...,1,0,0,1,0,0,0,1,0,Center_west_of_including_aristotle_square
2,Ρωμαϊκή Αγορά - Βενιζέλου - Ιστορικό Κέντρο (Κ...,39,173000,Studio / Γκαρσονιέρα,4,1.0,1.0,07/06/2025,1952,0,...,0,0,0,0,0,0,0,1,0,out_of_thessaloniki
3,Διοικητήριο (Κέντρο Θεσσαλονίκης),30,132000,Studio / Γκαρσονιέρα,4,1.0,1.0,07/06/2025,1952,0,...,1,0,0,1,0,0,0,0,0,Center_west_of_including_aristotle_square
4,Διοικητήριο (Κέντρο Θεσσαλονίκης),39,173000,Studio / Γκαρσονιέρα,4,1.0,1.0,07/06/2025,1952,1,...,1,0,0,1,0,1,0,0,0,Center_west_of_including_aristotle_square
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24309,Κάτω Τούμπα (Τούμπα),160,496000,Μεζονέτα,5,3.0,1.0,09/02/2024,2022,0,...,0,0,1,0,0,0,0,1,1,Toumpa
24310,Δελασάλ (Πεύκα),140,260000,Μεζονέτα,1,3.0,1.0,28/11/2023,2022,1,...,0,0,0,0,1,0,0,1,0,Peuka
24311,Άνωθεν Περιφερειακού (Εύοσμος),135,240000,Μεζονέτα,4,3.0,1.0,27/01/2025,2022,0,...,0,0,0,0,1,0,0,1,1,Evosmos_Kordelio
24312,Επέκταση (Πυλαία),270,500000,Μονοκατοικία,4,5.0,1.0,17/10/2024,2022,1,...,1,0,1,0,1,0,0,1,1,Pylaia


We see that the floor column contains only numbers now.

In [16]:
df_clean["Floor"].unique()

array(['7', '4', '3', '6', '8', '1', 0.5, -0.5, -1, '5', '2', 0, '10',
       '9', '18', '16', '12', 3, 2], dtype=object)

In [17]:
def fix_types(df):
    df["Price (in €)"] = df["Price (in €)"].astype(float)
    df["Total_area (square meters)"] = df["Total_area (square meters)"].astype(float)
    df["Floor"] = df["Floor"].astype(float)
    df["submission_date"] = pd.to_datetime(df["submission_date"],format="%d/%m/%Y")
    return df
df_clean = fix_types(df_clean)

Finally we save the dataframe as a csv file.

In [19]:
df_clean.to_csv("./Houses_final.csv",index=False)